In [52]:
import pandas as pd
import numpy as np

In [53]:
df = pd.read_csv("../sp500_ohlcv_2005_2025_2.csv")

In [54]:

# Suppose df a les colonnes : Date, Ticker, Open, High, Low, Close, Volume
df = df.sort_values(["Ticker", "Date"])

# --- 1) Log-return
df["log_return"] = np.log(df["Close"]) - np.log(df["Close"].shift(1))

# --- 2) Target direction (r_{t+1} > 0)
df["target"] = (df.groupby("Ticker")["log_return"].shift(-1) > 0).astype(int)

# --- 3) Features momentum
df["mom_5"]  = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(5) - 1)
df["mom_21"] = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(21) - 1)

# --- 4) Features volatilité
df["vol_5"]  = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(5).std())
df["vol_21"] = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(21).std())

# --- 5) High–Low range
df["range"] = (df["High"] - df["Low"]) / df["Open"]

# --- 6) Volume z-score
df["volume_z"] = df.groupby("Ticker")["Volume"].transform(
    lambda x: (x - x.mean()) / x.std()
)

df['Return'] = df.groupby('Ticker')['Close'].pct_change(fill_method=None)

# --- 7) Clean
df = df.dropna()


In [55]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0"}

resp = requests.get(url, headers=headers)
resp.raise_for_status()  # lève une erreur si 4xx/5xx

tables = pd.read_html(resp.text, header=0)
sp500 = tables[0]

sp500 = sp500.rename(columns={"Symbol": "Ticker", "GICS Sector": "Sector"})
sp500['Ticker'] = sp500['Ticker'].str.replace('.', '-', regex=False)

df = df.sort_values(by=['Ticker', 'Date'])

df = df.merge(
    sp500[['Ticker', 'Sector']], 
    on='Ticker', 
    how='left'
)

df.tail()

/tmp/ipykernel_56735/543082282.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text, header=0)


,Date,Ticker,Open,High,Low,Close,Volume,log_return,target,mom_5,mom_21,vol_5,vol_21,range,volume_z,Return,Sector
2278836,2024-12-24,ZTS,161.553123,162.875546,160.605722,162.540009,1023600.0,0.002553,1,-0.035658,-0.069281,0.013838,0.011887,0.014050,-0.714869,0.002557,Health Care
2278837,2024-12-26,ZTS,161.572877,163.615722,160.882060,163.349274,2167200.0,0.004967,0,-0.008921,-0.073807,0.011590,0.011660,0.016919,-0.256480,0.004979,Health Care
2278838,2024-12-27,ZTS,162.786715,164.345996,161.375477,162.441315,1800100.0,-0.005574,0,0.006605,-0.063176,0.005697,0.011267,0.018248,-0.403625,-0.005558,Health Care
2278839,2024-12-30,ZTS,161.740613,161.898519,159.332611,160.112259,1531400.0,-0.014442,1,-0.015773,-0.082041,0.007613,0.011328,0.015864,-0.511328,-0.014338,Health Care
2278840,2024-12-31,ZTS,160.763608,161.602466,159.747117,160.793213,1327400.0,0.004244,0,-0.008218,-0.070300,0.008295,0.011421,0.011541,-0.593097,0.004253,Health Care


la variable Sector est catégorielle par conséquent on va l'encoder par par la volatilité moyenne du secteur. Ce choix est motivé par la raison suivante. On veut faire comprendre au modèle que certains secteurs sont très volatils et très dépendants de chocs extérieur ainsi pour ces secteurs la, l'évolution futur de l'action est moins évidente.

In [56]:
# 1) Calcul de volatilité moyenne par secteur
sector_vol_mean = df.groupby("Sector")["vol_21"].mean()

# 2) Encodage de Sector par cette moyenne
df["Sector_encoded"] = df["Sector"].map(sector_vol_mean)


In [57]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def classification_logreg(df, features, threshold=0.5):
    """
    Classification binaire via régression logistique, avec split temporel et métriques globales
    + métrique par secteur.

    Split temporel (chronologique) :
      - train : Date < 2018-01-01
      - val   : 2018-01-01 <= Date < 2021-01-01  (préparé mais non utilisé ici)
      - test  : Date >= 2021-01-01

    Le modèle apprend P(target=1 | X). On convertit ensuite en classe via un seuil `threshold`.

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "target", "Sector" + les colonnes de `features`.
        Ici `target` doit être binaire (0/1).
    features : list[str]
        Colonnes explicatives.
    threshold : float, par défaut 0.5
        Seuil de décision : y_pred = 1 si proba > threshold.

    Retours
    -------
    model : LogisticRegression
        Modèle entraîné.
    sector_accuracy : pd.Series
        Accuracy par secteur, triée décroissante.
    """

    # Copie défensive pour ne pas modifier df
    df = df.copy()

    # ----------------------------------------
    # 1) Split train/val/test (chronologique)
    # ----------------------------------------
    train = df[df["Date"] < "2018-01-01"]
    val   = df[(df["Date"] >= "2018-01-01") & (df["Date"] < "2021-01-01")]
    test  = df[df["Date"] >= "2021-01-01"].copy()  # copy important car on ajoute "pred"

    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]   # non utilisé ici (mais utile pour tuning/threshold)
    X_test,  y_test  = test[features],  test["target"]

    # ----------------------------------------
    # 2) Entraînement du modèle
    # ----------------------------------------
    # max_iter augmenté pour éviter les non-convergences sur gros jeux de données.
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # ----------------------------------------
    # 3) Prédiction : proba puis classe via seuil
    # ----------------------------------------
    # predict_proba renvoie [P(class=0), P(class=1)] ; on garde la proba de la classe 1
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Binarisation selon le seuil
    y_pred = (y_pred_proba > threshold).astype(int)

    # ----------------------------------------
    # 4) Métriques globales
    # ----------------------------------------
    # Accuracy globale sur le test
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # (Optionnel mais souvent utile en finance / classes déséquilibrées)
    # - AUC : qualité de ranking des probabilités (indépendant du seuil)
    # - F1  : compromis précision/rappel (utile si classe 1 rare)
    # - confusion matrix / report : diagnostic complet
    # On ne print pas tout par défaut, mais tu peux décommenter au besoin.
    # auc = roc_auc_score(y_test, y_pred_proba)
    # f1  = f1_score(y_test, y_pred)
    # print("AUC      :", auc)
    # print("F1       :", f1)
    # print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

    # On stocke la prédiction dans test pour calculer des métriques par groupe
    test["pred"] = y_pred

    # ----------------------------------------
    # 5) Métrique par secteur
    # ----------------------------------------
    sector_accuracy = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return model, sector_accuracy


In [58]:
regression(df, ["mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])


Accuracy : 0.5177679134039983


(LogisticRegression(max_iter=1000),
 Sector
 Financials                0.524993
 Utilities                 0.521979
 Industrials               0.521291
 Real Estate               0.520815
 Energy                    0.518289
 Information Technology    0.516843
 Consumer Staples          0.516080
 Consumer Discretionary    0.515701
 Health Care               0.511678
 Communication Services    0.509799
 Materials                 0.506109
 dtype: float64)

De premier abord, la regression parait ne pas être suffisante pour prédire le signe du rendement

In [59]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def training_xgb_classifier(df, features, threshold=0.5):
    """
    Entraîne un classifieur XGBoost (XGBClassifier) sur une cible binaire `target`,
    avec un split temporel (train/val/test), puis évalue sur le test et calcule une
    métrique par secteur.

    Split temporel :
      - train : Date < 2018-01-01
      - val   : 2018-01-01 <= Date < 2021-01-01   (utilisé pour le suivi pendant fit)
      - test  : Date >= 2021-01-01

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "target", "Sector" + les colonnes de `features`.
        `target` doit être binaire (0/1).
    features : list[str]
        Colonnes explicatives.
    threshold : float, par défaut 0.5
        Seuil de décision sur la probabilité de classe 1.

    Retours
    -------
    sector_accuracy : pd.Series
        Accuracy par secteur sur le test (triée décroissante).
    """

    # Copie défensive pour ne pas modifier l'objet df en entrée
    df = df.copy()

    # ----------------------------------------
    # 1) Split train/val/test (chronologique)
    # ----------------------------------------
    train = df[df["Date"] < "2018-01-01"]
    val   = df[(df["Date"] >= "2018-01-01") & (df["Date"] < "2021-01-01")]
    test  = df[df["Date"] >= "2021-01-01"].copy()  # copy car on ajoute ensuite "pred"

    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ----------------------------------------
    # 2) Modèle XGBoost (classification)
    # ----------------------------------------
    # Hyperparamètres :
    # - max_depth : complexité des arbres
    # - learning_rate : shrinkage (plus faible => + d'arbres)
    # - n_estimators : nombre max d'arbres (pas d'early stopping ici)
    # - subsample / colsample_bytree : bagging pour régulariser
    #
    # NB : tu peux aussi fixer random_state pour reproductibilité.
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss",  # évite warnings et définit la métrique suivie sur eval_set
    )

    # Entraînement (val fourni pour monitoring, mais sans early stopping ici)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ----------------------------------------
    # 3) Prédiction (proba -> classe via seuil)
    # ----------------------------------------
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    # ----------------------------------------
    # 4) Métriques globales (test)
    # ----------------------------------------
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # Optionnel : utile si classes déséquilibrées
    # f1  = f1_score(y_test, y_pred)
    # auc = roc_auc_score(y_test, y_pred_proba)
    # print("F1       :", f1)
    # print("AUC      :", auc)
    # print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

    # Ajout des prédictions pour calculs par groupe
    test["pred"] = y_pred

    # ----------------------------------------
    # 5) Métrique par secteur
    # ----------------------------------------
    sector_accuracy = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return sector_accuracy


In [60]:
training(df, ["Close", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])

Accuracy : 0.5128854497365115


Sector
Financials                0.518164
Utilities                 0.517800
Real Estate               0.516996
Information Technology    0.515275
Materials                 0.515144
Industrials               0.512890
Energy                    0.512804
Consumer Staples          0.511582
Communication Services    0.507419
Health Care               0.506284
Consumer Discretionary    0.506053
dtype: float64

meme en ajoutant des variable pertinentes, l'accuracy reste sensiblement la meme peu importe le secteur. L'accuracy réalisée est 0.51. Peut etre qu'étant donné le fait que les années d'entrainement sont très anciennes par rapport aux années de test le modèle est mal adapté aux nouvelles années

Changeons de méthode d'entrainement : éparpillons des années de test et de validation dans tous le dataset

# Ajout de la variable secteur

In [61]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def training2_xgb_classifier(df, features, threshold=0.5):
    """
    Variante de training XGBoost classification avec un split "en blocs alternés" :
    on alterne des fenêtres temporelles pour train / val / test sur plusieurs périodes.

    Objectif typique :
    - évaluer la robustesse du modèle sur plusieurs régimes de marché (plusieurs cycles),
      plutôt qu'un seul bloc train->test.
    - éviter que tout le test soit concentré sur une seule période (ex: uniquement 2021+).

    Train :
      - 2004-01-01 .. 2008-01-01
      - 2010-01-01 .. 2013-01-01
      - 2015-01-01 .. 2018-01-01
      - 2020-01-01 .. 2023-01-01

    Val :
      - 2008-01-01 .. 2009-01-01
      - 2013-01-01 .. 2014-01-01
      - 2018-01-01 .. 2019-01-01
      - 2023-01-01 .. 2024-01-01

    Test :
      - 2009-01-01 .. 2010-01-01
      - 2014-01-01 .. 2015-01-01
      - 2019-01-01 .. 2020-01-01
      - 2024-01-01 .. 2025-01-01

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "target", "Sector" + features.
        `target` doit être binaire (0/1).
    features : list[str]
        Colonnes explicatives.
    threshold : float, par défaut 0.5
        Seuil proba -> classe.

    Retours
    -------
    sector_accuracy : pd.Series
        Accuracy par secteur sur l'ensemble test (toutes fenêtres confondues), triée décroissante.
    """

    df = df.copy()

    # ------------------------------------------------------------
    # 1) Construction des masques temporels (train / val / test)
    # ------------------------------------------------------------
    # .between(a, b) est inclusif des deux côtés (>=a et <=b).
    # Ici, ça peut créer des chevauchements aux bornes si tu n'es pas attentif.
    # (ex: une date = "2008-01-01" appartient à la fois à train et val)
    #
    # Si tu veux éviter *tout* overlap, il faut rendre les bornes disjointes
    # (par ex. utiliser < sur la borne haute).
    mask_train = (
        df["Date"].between("2004-01-01", "2008-01-01") |
        df["Date"].between("2010-01-01", "2013-01-01") |
        df["Date"].between("2015-01-01", "2018-01-01") |
        df["Date"].between("2020-01-01", "2023-01-01")
    )

    mask_val = (
        df["Date"].between("2008-01-01", "2009-01-01") |
        df["Date"].between("2013-01-01", "2014-01-01") |
        df["Date"].between("2018-01-01", "2019-01-01") |
        df["Date"].between("2023-01-01", "2024-01-01")
    )

    mask_test = (
        df["Date"].between("2009-01-01", "2010-01-01") |
        df["Date"].between("2014-01-01", "2015-01-01") |
        df["Date"].between("2019-01-01", "2020-01-01") |
        df["Date"].between("2024-01-01", "2025-01-01")
    )

    train = df.loc[mask_train]
    val   = df.loc[mask_val]
    test  = df.loc[mask_test].copy()  # copy car on ajoute "pred" ensuite

    # ------------------------------------------------------------
    # 2) Matrices X / y
    # ------------------------------------------------------------
    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ------------------------------------------------------------
    # 3) Modèle XGBoost (classification)
    # ------------------------------------------------------------
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss",
    )

    # Entraînement (val pour monitoring, sans early stopping explicite ici)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ------------------------------------------------------------
    # 4) Prédiction sur test
    # ------------------------------------------------------------
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    # Stockage pour métriques par groupe
    test["pred"] = y_pred

    # ------------------------------------------------------------
    # 5) Métriques globales
    # ------------------------------------------------------------
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # Optionnel
    # f1  = f1_score(y_test, y_pred)
    # auc = roc_auc_score(y_test, y_pred_proba)
    # print("F1       :", f1)
    # print("AUC      :", auc)
    # print(confusion_matrix(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

    # ------------------------------------------------------------
    # 6) Métrique par secteur
    # ------------------------------------------------------------
    sector_accuracy = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return sector_accuracy


In [62]:
training2(df, ["mom_21", "vol_21", "range", "volume_z", "Sector_encoded"])


Accuracy : 0.5285094832442767


Sector
Real Estate               0.541369
Utilities                 0.538882
Materials                 0.535286
Financials                0.535254
Industrials               0.529979
Information Technology    0.527850
Consumer Staples          0.524530
Health Care               0.521683
Communication Services    0.518975
Consumer Discretionary    0.518429
Energy                    0.516915
dtype: float64

la meilleur accuracy est meilleure que celle obtenue précédemment 0.5293 mais reste cependant faible. 

Essayons de préciser nos modèles en les concentrant sur un secteur. Peut être que les modèles précédent essayaient de trop généraliser ce qui les rendait imprécis.

In [63]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def training3_xgb_classifier_sector(df, features, sector, threshold=0.5):
    """
    Entraîne un XGBoost classifier *uniquement* sur un secteur donné, en utilisant
    un split temporel "en blocs alternés" (train/val/test sur plusieurs fenêtres).

    Différence avec `training2` :
    - Ici on filtre d'abord (implicitement via les masques) sur `Sector == sector`,
      donc le modèle est spécifique à un secteur.

    Train :
      - 2004-01-01 .. 2008-01-01
      - 2010-01-01 .. 2013-01-01
      - 2015-01-01 .. 2018-01-01
      - 2020-01-01 .. 2023-01-01

    Val :
      - 2008-01-01 .. 2009-01-01
      - 2013-01-01 .. 2014-01-01
      - 2018-01-01 .. 2019-01-01
      - 2023-01-01 .. 2024-01-01

    Test :
      - 2009-01-01 .. 2010-01-01
      - 2014-01-01 .. 2015-01-01
      - 2019-01-01 .. 2020-01-01
      - 2024-01-01 .. 2025-01-01

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "Sector", "target" + features.
        `target` doit être binaire (0/1).
    features : list[str]
        Variables explicatives.
    sector : str
        Valeur de la colonne "Sector" à sélectionner (ex: "Information Technology").
    threshold : float, par défaut 0.5
        Seuil proba -> classe.

    Retours
    -------
    metrics : dict
        Dictionnaire de métriques utiles pour ce secteur :
        - "sector" : secteur
        - "accuracy" : accuracy globale sur test
        - "n_train", "n_val", "n_test" : tailles des splits
        - "model" : modèle entraîné
    """

    df = df.copy()

    # ------------------------------------------------------------
    # 1) Masques temporels + filtre sector
    # ------------------------------------------------------------
    mask_train = (
        (df["Sector"] == sector) &
        (
            df["Date"].between("2004-01-01", "2008-01-01") |
            df["Date"].between("2010-01-01", "2013-01-01") |
            df["Date"].between("2015-01-01", "2018-01-01") |
            df["Date"].between("2020-01-01", "2023-01-01")
        )
    )

    mask_val = (
        (df["Sector"] == sector) &
        (
            df["Date"].between("2008-01-01", "2009-01-01") |
            df["Date"].between("2013-01-01", "2014-01-01") |
            df["Date"].between("2018-01-01", "2019-01-01") |
            df["Date"].between("2023-01-01", "2024-01-01")
        )
    )

    mask_test = (
        (df["Sector"] == sector) &
        (
            df["Date"].between("2009-01-01", "2010-01-01") |
            df["Date"].between("2014-01-01", "2015-01-01") |
            df["Date"].between("2019-01-01", "2020-01-01") |
            df["Date"].between("2024-01-01", "2025-01-01")
        )
    )

    train = df.loc[mask_train]
    val   = df.loc[mask_val]
    test  = df.loc[mask_test].copy()

    # ------------------------------------------------------------
    # 2) Matrices X / y
    # ------------------------------------------------------------
    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ------------------------------------------------------------
    # 3) Modèle XGBoost (classification)
    # ------------------------------------------------------------
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss",
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ------------------------------------------------------------
    # 4) Prédictions et métriques (test)
    # ------------------------------------------------------------
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # Optionnel :
    # f1  = f1_score(y_test, y_pred)
    # auc = roc_auc_score(y_test, y_pred_proba)

    return {
        "sector": sector,
        "accuracy": acc,
        "n_train": len(train),
        "n_val": len(val),
        "n_test": len(test),
        "model": model,
        # "f1": f1,
        # "auc": auc,
    }


In [64]:
training3(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "Utilities")

Accuracy : 0.5264772575696945


Sector
Utilities    0.526477
dtype: float64

In [65]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def training4_xgb_classifier_ticker(df, features, ticker, threshold=0.5):
    """
    Entraîne un XGBoost classifier *uniquement* sur un ticker donné, avec un split temporel
    "en blocs alternés" (plusieurs fenêtres train/val/test).

    Train :
      - 2004-01-01 .. 2008-01-01
      - 2010-01-01 .. 2013-01-01
      - 2015-01-01 .. 2018-01-01
      - 2020-01-01 .. 2023-01-01

    Val :
      - 2008-01-01 .. 2009-01-01
      - 2013-01-01 .. 2014-01-01
      - 2018-01-01 .. 2019-01-01
      - 2023-01-01 .. 2024-01-01

    Test :
      - 2009-01-01 .. 2010-01-01
      - 2014-01-01 .. 2015-01-01
      - 2019-01-01 .. 2020-01-01
      - 2024-01-01 .. 2025-01-01

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "Ticker", "target" + features.
        `target` doit être binaire (0/1).
    features : list[str]
        Colonnes explicatives.
    ticker : str
        Ticker à sélectionner (ex: "AAPL").
    threshold : float, par défaut 0.5
        Seuil de décision sur la probabilité de la classe 1.

    Retours
    -------
    results : dict
        Résultats centrés sur CE ticker :
        - "ticker" : ticker
        - "accuracy" : accuracy globale sur test
        - "n_train", "n_val", "n_test" : tailles
        - "model" : modèle entraîné

    Notes importantes
    -----------------
    """

    df = df.copy()

    # ------------------------------------------------------------
    # 1) Masques temporels + filtre ticker
    # ------------------------------------------------------------
    mask_train = (
        (df["Ticker"] == ticker) &
        (
            df["Date"].between("2004-01-01", "2008-01-01") |
            df["Date"].between("2010-01-01", "2013-01-01") |
            df["Date"].between("2015-01-01", "2018-01-01") |
            df["Date"].between("2020-01-01", "2023-01-01")
        )
    )

    mask_val = (
        (df["Ticker"] == ticker) &
        (
            df["Date"].between("2008-01-01", "2009-01-01") |
            df["Date"].between("2013-01-01", "2014-01-01") |
            df["Date"].between("2018-01-01", "2019-01-01") |
            df["Date"].between("2023-01-01", "2024-01-01")
        )
    )

    mask_test = (
        (df["Ticker"] == ticker) &
        (
            df["Date"].between("2009-01-01", "2010-01-01") |
            df["Date"].between("2014-01-01", "2015-01-01") |
            df["Date"].between("2019-01-01", "2020-01-01") |
            df["Date"].between("2024-01-01", "2025-01-01")
        )
    )

    train = df.loc[mask_train]
    val   = df.loc[mask_val]
    test  = df.loc[mask_test].copy()

    # ------------------------------------------------------------
    # 2) Matrices X / y
    # ------------------------------------------------------------
    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ------------------------------------------------------------
    # 3) Modèle XGBoost (classification)
    # ------------------------------------------------------------
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss",
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ------------------------------------------------------------
    # 4) Prédictions et métriques (test)
    # ------------------------------------------------------------
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # Optionnel :
    # f1  = f1_score(y_test, y_pred)
    # auc = roc_auc_score(y_test, y_pred_proba)

    return {
        "ticker": ticker,
        "accuracy": acc,
        "n_train": len(train),
        "n_val": len(val),
        "n_test": len(test),
        "model": model,
        # "f1": f1,
        # "auc": auc,
    }


In [66]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "GOOGL")

Accuracy : 0.5099206349206349


Sector
Communication Services    0.509921
dtype: float64

In [67]:
training4(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"], "XOM")


Accuracy : 0.5277777777777778


Sector
Energy    0.527778
dtype: float64

On remarque que souvent la catégories la moins bien prédites est celle de l'énergie qui est le secteur le plus sensible au choc d'après ce qui a été vu précédemment. Peut être que le faible score provient des période de crises dues à des chocs extérieurs aux marché qui sont imprévisibles à l'aide 'uniquement les données. Essayons de se limiter à la période la plus calme de 2005-2025

In [68]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report


def training5_xgb_classifier(df, features, threshold=0.5):
    """
    Entraîne un XGBoost classifier sur une fenêtre temporelle courte, avec split simple :

      - train : 2010-01-01 .. 2012-01-01
      - val   : 2012-01-01 .. 2013-01-01
      - test  : 2013-01-01 .. 2014-01-01

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "target", "Sector" + features.
        `target` doit être binaire (0/1).
    features : list[str]
        Colonnes explicatives.
    threshold : float, par défaut 0.5
        Seuil de décision pour convertir la proba en classe.

    Retours
    -------
    sector_accuracy : pd.Series
        Accuracy par secteur sur le test (triée décroissante).
        (Ton code original appelle ça sector_f1 mais calcule accuracy_score.)

    Remarques
    ---------
    - `between(a, b)` inclut les bornes => "2012-01-01" peut être dans train ET val,
      et "2013-01-01" peut être dans val ET test. Si tu veux éviter tout overlap,
      préfère des intervalles [a, b) (borne haute exclusive).
    """

    df = df.copy()

    # ------------------------------------------------------------
    # 1) Split temporel (attention aux bornes inclusives)
    # ------------------------------------------------------------
    mask_train = df["Date"].between("2010-01-01", "2012-01-01")
    mask_val   = df["Date"].between("2012-01-01", "2013-01-01")
    mask_test  = df["Date"].between("2013-01-01", "2014-01-01")

    train = df.loc[mask_train]
    val   = df.loc[mask_val]
    test  = df.loc[mask_test].copy()

    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ------------------------------------------------------------
    # 2) Modèle XGBoost (classification)
    # ------------------------------------------------------------
    model = xgb.XGBClassifier(
        max_depth=5,
        learning_rate=0.03,
        n_estimators=800,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss",
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ------------------------------------------------------------
    # 3) Prédictions sur test
    # ------------------------------------------------------------
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    # Stockage pour métriques par groupe
    test["pred"] = y_pred

    # ------------------------------------------------------------
    # 4) Métrique globale
    # ------------------------------------------------------------
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy :", acc)

    # ------------------------------------------------------------
    # 5) Métrique par secteur
    # ------------------------------------------------------------
    # Attention : ton code l'appelle sector_f1 mais calcule accuracy_score.
    sector_accuracy = (
        test.groupby("Sector")
            .apply(lambda g: accuracy_score(g["target"], g["pred"]), include_groups=False)
            .sort_values(ascending=False)
    )

    return sector_accuracy


In [69]:
training5(df, ["mom_5", "vol_5", "range", "volume_z", "mom_21"])


Accuracy : 0.5262041088323235


Sector
Materials                 0.541235
Industrials               0.531409
Consumer Staples          0.530544
Health Care               0.529594
Financials                0.529561
Information Technology    0.527334
Consumer Discretionary    0.525391
Communication Services    0.520982
Utilities                 0.516694
Energy                    0.513228
Real Estate               0.505200
dtype: float64

Malheuresement les résultats ne sont pas sensiblement meilleur

## Interprétation d’une accuracy proche de 51 % et intérêt d’un passage de la classification à la régression

### Contexte
On considère un problème de classification binaire où l’on cherche à prédire le signe du rendement futur :
$$
y_{t+h} = \mathbb{1}_{\{r_{t+h} > 0\}},
$$
à partir d’un vecteur de caractéristiques $X_t$. Les différents modèles testés (régression logistique, classifieurs XGBoost, variantes de découpages temporels) conduisent à une accuracy hors-échantillon proche de 51 %.

### 1) Que signifie une accuracy proche de 51 % ?
Une accuracy d’environ 51 % doit être interprétée relativement à une référence. Lorsque la proportion de rendements positifs est proche de 50 %, une stratégie aléatoire ou un classifieur constant (par exemple « toujours positif ») obtient typiquement une accuracy autour de 50 %. Un résultat à 51 % indique donc un pouvoir prédictif très faible sur le signe du rendement, ce qui est cohérent avec plusieurs faits stylisés en finance :

- Faible prévisibilité du signe à court horizon. Les rendements à fréquence journalière ou hebdomadaire sont dominés par du bruit et par des chocs difficiles à anticiper.
- Instabilité de l’étiquette au voisinage de zéro. Une part importante des rendements est proche de 0, si bien qu’une variation marginale (ou un bruit de mesure) peut faire basculer l’étiquette de 0 à 1, rendant la variable cible fragile.
- Dépendances temporelles et régimes de marché. Le caractère non stationnaire (changements de volatilité, cycles, crises) réduit la stabilité de la relation $X_t \mapsto y_{t+h}$ hors-échantillon.

Ainsi, une accuracy proche de 50 % n’implique pas nécessairement que les caractéristiques $X_t$ sont inutiles en toute généralité ; elle suggère plutôt que, sous cette formulation, le signe du rendement est un objet particulièrement difficile à prédire et que l’information contenue dans $X_t$ est au mieux marginale à l’horizon étudié.

### 2) Limites conceptuelles du cadrage « classification : rendement positif ou non »
Transformer un rendement continu $r_{t+h}$ en une variable binaire $y_{t+h}$ réduit fortement l’information disponible.

1. Perte de l’information de magnitude. Les observations $r = 0.01\%$ et $r = 3\%$ reçoivent le même label $y=1$, alors que leurs implications économiques sont très différentes. De même, $r = -0.01\%$ et $r = -4\%$ reçoivent le même label $y=0$. Or la magnitude des mouvements influence directement performance et risque.

2. Seuil de décision économiquement arbitraire. Le seuil 0 n’intègre pas les coûts de transaction (spread, commission, slippage). Un rendement légèrement positif peut être non exploitable une fois les coûts pris en compte. Optimiser une fonction de perte centrée sur $\mathbb{1}_{\{r_{t+h}>0\}}$ peut donc être mal aligné avec un objectif économique.

3. Métrique d’évaluation potentiellement inadéquate. L’accuracy pénalise de façon identique toutes les erreurs, indépendamment de l’ampleur du mouvement manqué. En finance, se tromper sur un mouvement extrême a généralement un coût économique plus élevé que se tromper sur une variation quasi nulle.

### 3) Pourquoi reformuler le problème en régression peut être plus pertinent
Une alternative consiste à prédire directement le rendement (ou une transformation du rendement) :
$$
\hat r_{t+h} \approx \mathbb{E}\!\left[r_{t+h}\mid X_t\right].
$$
Cette reformulation présente plusieurs avantages.

1. Alignement avec l’objectif économique. L’objet central en allocation d’actifs est l’espérance conditionnelle du rendement (éventuellement ajustée du risque). Une prédiction continue $\hat r_{t+h}$ peut être utilisée comme score pour construire des positions (pondérations proportionnelles au signal, ou sélection via ranking), ce qui correspond davantage à une logique de portefeuille.

2. Exploitation de l’information de magnitude. La régression conserve la granularité des valeurs : le modèle est incité à distinguer des scénarios économiquement différents, plutôt que de uniquement franchir un seuil.

3. Règles de décision plus robustes que « $r>0$ ». À partir d’un score continu, il est possible d’éviter la zone la plus bruitée autour de zéro, par exemple :
- prendre des positions uniquement sur les quantiles extrêmes (top et bottom),
- imposer un seuil $c>0$ reflétant coûts et marge de sécurité, via une règle du type $\hat r_{t+h} > c$,
- utiliser un score ajusté du risque, par exemple $\hat r_{t+h} / \hat\sigma_{t+h}$, où $\hat\sigma_{t+h}$ est une prévision de volatilité.

4. Évaluation mieux adaptée via des métriques de ranking et de portefeuille. En pratique, l’utilité d’un modèle de rendements est souvent mieux reflétée par :
- des mesures de corrélation de rang (par exemple corrélation de Spearman) entre $\hat r_{t+h}$ et $r_{t+h}$,
- la performance d’un portefeuille construit à partir des scores (exemple : long sur le top quantile et short sur le bottom quantile),
- des statistiques de risque (volatilité, drawdown) et de turnover.
Ces critères sont souvent plus informatifs que l’accuracy binaire lorsque le signal est faible mais économiquement exploitable.

### 4) Remarque : la régression ne garantit pas un $R^2$ élevé
Prédire des rendements reste difficile et l’on observe fréquemment des $R^2$ faibles hors-échantillon. Cependant, un $R^2$ faible n’exclut pas une utilité économique si le modèle améliore le classement relatif des actifs ou la prise de décision sur les extrêmes, ce qui est souvent l’objectif opérationnel.

### Conclusion
Une accuracy proche de 51 % sur la prédiction du signe du rendement suggère que, dans cette formulation, le problème est dominé par le bruit et que le seuil $r_{t+h}>0$ est peu informatif et possiblement mal aligné avec l’objectif économique. Reformuler la tâche en régression (prédire la valeur du rendement) ou, plus généralement, en problème de scoring/ranking permet de conserver l’information de magnitude, de définir des règles de décision plus robustes (quantiles, seuils tenant compte des coûts) et d’évaluer les modèles selon des critères plus cohérents avec la construction d’un portefeuille.
